## Import Libraries

In [0]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### We need to download these dictionaries first to perform the operations

In [0]:
nltk.download('stopwords')
## for removing the stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
train = pd.read_csv('/content/drive/My Drive/Shaastra Workshop Material/Text Analysis/train_E6oV3lV.csv', index_col='id')

In [0]:
pd.set_option('display.width',100000000)

In [0]:
train.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


In [0]:
train.shape

(31962, 2)

# Basic feature extraction

## Number of Words

One of the most basic features we can extract is the number of words in each tweet. The basic intuition behind this is that generally, the negative sentiments contain a lesser amount of words than the positive ones.



train['word_count'] = train['tweet'].apply(lambda x: len(str(x).split(" ")))
train[['word_count','tweet']].head()
    

## Number of characters

In [0]:
train['character_count']=train['tweet'].apply(lambda x: len(x))
#alternate
#train['word_count'] = train['tweet'].str.len()
train.head()

,label,tweet,character_count
id,,,
1,0,@user when a father is dysfunctional and is s...,102
2,0,@user @user thanks for #lyft credit i can't us...,122
3,0,bihday your majesty,21
4,0,#model i love u take with u all the time in ...,86
5,0,factsguide: society now #motivation,39


## Average word length

In [0]:
#train['avg_word_len']=train['character_count']/train['word_count']
#this works but is not right since character_count also includes spaces

def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['tweet'].apply(lambda x: avg_word(x))
train[['tweet','avg_word']].head()

,tweet,avg_word
id,,
1,@user when a father is dysfunctional and is s...,4.555556
2,@user @user thanks for #lyft credit i can't us...,5.315789
3,bihday your majesty,5.666667
4,#model i love u take with u all the time in ...,4.928571
5,factsguide: society now #motivation,8.000000


## Counting number of stopwords
Stopwords are words which are filtered out before processing of natural language data. Stop words are generally the most common words in a language. Example, the, what, where, how, has, had etc.

Generally, while solving an NLP problem, the first thing we do is to remove the stopwords. But sometimes calculating the number of stopwords can also give us some extra information which we might have been losing before.

For this we use a library called nltk, a basic NLP library

In [0]:
stop = stopwords.words('english')

train['stopwords'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['tweet','stopwords']].head()

,tweet,stopwords
id,,
1,@user when a father is dysfunctional and is s...,10
2,@user @user thanks for #lyft credit i can't us...,5
3,bihday your majesty,1
4,#model i love u take with u all the time in ...,5
5,factsguide: society now #motivation,1


## Number of special characters
One more interesting feature which we can extract from a tweet is calculating the number of hashtags or mentions present in it. This also helps in extracting extra information from our text data.

Here, we make use of the ‘starts with’ function because hashtags (or mentions) always appear at the beginning of a word.



In [0]:
train['hastags'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['tweet','hastags']].head()

,tweet,hastags
id,,
1,@user when a father is dysfunctional and is s...,1
2,@user @user thanks for #lyft credit i can't us...,3
3,bihday your majesty,0
4,#model i love u take with u all the time in ...,1
5,factsguide: society now #motivation,1


 ## Number of numerics
 Just like we calculated the number of words, we can also calculate the number of numerics which are present in the tweets. It does not have a lot of use in our example, but this is still a useful feature that should be run while doing similar exercises. 



In [0]:
train['numerics'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train[['tweet','numerics']].head()

,tweet,numerics
id,,
1,@user when a father is dysfunctional and is s...,0
2,@user @user thanks for #lyft credit i can't us...,0
3,bihday your majesty,0
4,#model i love u take with u all the time in ...,0
5,factsguide: society now #motivation,0


## Number of Uppercase words
Anger or rage is quite often expressed by writing in UPPERCASE words which makes this a necessary operation to identify those words.

In [0]:
train['upper'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train[['tweet','upper']].head()

,tweet,upper
id,,
1,@user when a father is dysfunctional and is s...,0
2,@user @user thanks for #lyft credit i can't us...,0
3,bihday your majesty,0
4,#model i love u take with u all the time in ...,0
5,factsguide: society now #motivation,0


# Basic feature extraction

So far, we have learned how to extract basic features from text data. Before diving into text and feature extraction, our first step should be cleaning the data in order to obtain better features. We will achieve this by doing some of the basic pre-processing steps on our training data.

## Lower case
The first pre-processing step which we will do is transform our tweets into lower case. This avoids having multiple copies of the same words. For example, while calculating the word count, ‘Analytics’ and ‘analytics’ will be taken as different words.

In [0]:
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['tweet'].head()

id
1    @user when a father is dysfunctional and is so...
2    @user @user thanks for #lyft credit i can't us...
3                                  bihday your majesty
4    #model i love u take with u all the time in ur...
5                  factsguide: society now #motivation
Name: tweet, dtype: object

##  Removing Punctuation
The next step is to remove punctuation, as it doesn’t add any extra information while treating text data. Therefore removing all instances of it will help us reduce the size of the training data.

In [0]:
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')
train['tweet'].head()

id
1    user when a father is dysfunctional and is so ...
2    user user thanks for lyft credit i cant use ca...
3                                  bihday your majesty
4    model i love u take with u all the time in urð...
5                    factsguide society now motivation
Name: tweet, dtype: object

## Removal of Stop Words
As we discussed earlier, stop words (or commonly occurring words) should be removed from the text data. For this purpose, we can either create a list of stopwords ourselves or we can use predefined libraries.

In [0]:
nltk.download('stopwords')
## for removing the stopwords
from nltk.corpus import stopwords

stop = stopwords.words('english')
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['tweet'].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


id
1    user father dysfunctional selfish drags kids d...
2    user user thanks lyft credit cant use cause do...
3                                       bihday majesty
4                model love u take u time urð ðððð ððð
5                        factsguide society motivation
Name: tweet, dtype: object

## Common word removal
Previously, we just removed commonly occurring words in a general sense. We can also remove commonly occurring words from our text data First, let’s check the 10 most frequently occurring words in our text data then take call to remove or retain.

In [0]:
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()[:1]
freq

user    17473
dtype: int64

In [0]:
#Now, let’s remove these words as their presence will not of any use in classification of our text data.

freq = list(freq.index)
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['tweet'].head()

id
1    father dysfunctional selfish drags kids dysfun...
2    thanks lyft credit cant use cause dont offer w...
3                                       bihday majesty
4                model love u take u time urð ðððð ððð
5                        factsguide society motivation
Name: tweet, dtype: object

## Rare words removal
Similarly, just as we removed the most common words, this time let’s remove rarely occurring words from the text. Because they’re so rare, the association between them and other words is dominated by noise. You can replace rare words with a more general form and then this will have higher counts



In [0]:
freq = pd.Series(' '.join(train['tweet']).split()).value_counts()[-10:]
freq

foiinãbranlable                         1
ibizabringitonmallorcaholidayssummer    1
quiltbag                                1
sunsetcliffs                            1
newsgod                                 1
pedophiles                              1
fetishes                                1
faketroll                               1
noballs                                 1
berlincitygirl                          1
dtype: int64

In [0]:
freq = list(freq.index)
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['tweet'].head()

id
1    father dysfunctional selfish drags kids dysfun...
2    thanks lyft credit cant use cause dont offer w...
3                                       bihday majesty
4                model love u take u time urð ðððð ððð
5                        factsguide society motivation
Name: tweet, dtype: object

## Spelling correction
We’ve all seen tweets with a plethora of spelling mistakes. Our timelines are often filled with hastly sent tweets that are barely legible at times.

In that regard, spelling correction is a useful pre-processing step because this also will help us in reducing multiple copies of words. For example, “Analytics” and “analytcs” will be treated as different words even if they are used in the same sense.

To achieve this we will use the textblob library. If you are not familiar with it, you can check my previous article on ‘NLP for beginners using textblob’.

In [0]:
from textblob import TextBlob
train['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

id
1    father dysfunctional selfish drags kiss dysfun...
2    thanks left credit can use cause dont offer wh...
3                                       midday majesty
4                 model love u take u time or ðððð ððð
5                        factsguide society motivation
Name: tweet, dtype: object

Note that it will actually take a lot of time to make these corrections. Therefore, just for the purposes of learning, I have shown this technique by applying it on only the first 5 rows. Moreover, we cannot always expect it to be accurate so some care should be taken before applying it.

We should also keep in mind that words are often used in their abbreviated form. For instance, ‘your’ is used as ‘ur’. We should treat this before the spelling correction step, otherwise these words might be transformed into any other word like the one shown below:



## Tokenization
Tokenization refers to dividing the text into a sequence of words or sentences. In our example, we have used the textblob library to first transform our tweets into a blob and then converted them into a series of words.

In [0]:
nltk.download('punkt')
TextBlob(train['tweet'][3]).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


WordList(['bihday', 'majesty'])

## Stemming
Stemming refers to the removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach. For this purpose, we will use PorterStemmer from the NLTK library.

In [0]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
train['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

id
1        father dysfunct selfish drag kid dysfunct run
2    thank lyft credit cant use caus dont offer whe...
3                                       bihday majesti
4                              model take urð ðððð ððð
5                              factsguid societi motiv
Name: tweet, dtype: object

## Lemmatization
Lemmatization is a more effective option than stemming because it converts the word into its root word, rather than just stripping the suffices. It makes use of the vocabulary and does a morphological analysis to obtain the root word. Therefore, we usually prefer using lemmatization over stemming.



In [0]:
from textblob import Word
nltk.download('wordnet')
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['tweet'].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


id
1    father dysfunctional selfish drag kid dysfunct...
2    thanks lyft credit cant use cause dont offer w...
3                                       bihday majesty
4                model love u take u time urð ðððð ððð
5                        factsguide society motivation
Name: tweet, dtype: object

# Advanced Text Processing
Up to this point, we have done all the basic pre-processing steps in order to clean our data. Now, we can finally move on to extracting features using NLP techniques.

 

## N-grams
N-grams are the combination of multiple words used together. Ngrams with N=1 are called unigrams. Similarly, bigrams (N=2), trigrams (N=3) and so on can also be used.

Unigrams do not usually contain as much information as compared to bigrams and trigrams. The basic principle behind n-grams is that they capture the language structure, like what letter or word is likely to follow the given one. The longer the n-gram (the higher the n), the more context you have to work with. Optimum length really depends on the application – if your n-grams are too short, you may fail to capture important differences. On the other hand, if they are too long, you may fail to capture the “general knowledge” and only stick to particular cases.

So, let’s quickly extract bigrams from our tweets using the ngrams function of the textblob library.



In [0]:
TextBlob(train['tweet'][1]).ngrams(2)

[WordList(['father', 'dysfunctional']),
 WordList(['dysfunctional', 'selfish']),
 WordList(['selfish', 'drag']),
 WordList(['drag', 'kid']),
 WordList(['kid', 'dysfunction']),
 WordList(['dysfunction', 'run'])]

## Term frequency
Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

Therefore, we can generalize term frequency as:

TF = (Number of times term T appears in the particular row) / (number of terms in that row)

In [0]:
tf1 = (train['tweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,lyft,1
1,offer,1
2,cant,1
3,pdx,1
4,thanks,1
5,dont,1
6,credit,1
7,van,1
8,getthanked,1
9,disapointed,1


## Inverse Document Frequency
The intuition behind inverse document frequency (IDF) is that a word is not of much use to us if it’s appearing in all the documents.

Therefore, the IDF of each word is the log of the ratio of the total number of rows to the number of rows in which that word is present.

IDF = log(N/n), where, N is the total number of rows and n is the number of rows in which the word was present.

So, let’s calculate IDF for the same tweets for which we calculated the term frequency.

In [0]:
for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['tweet'].str.contains(word)])))

tf1

,words,tf,idf
0,lyft,1,8.762865
1,offer,1,6.522155
2,cant,1,3.538194
3,pdx,1,8.762865
4,thanks,1,4.597751
5,dont,1,3.745585
6,credit,1,7.327781
7,van,1,5.236505
8,getthanked,1,9.679156
9,disapointed,1,10.372303


## Term Frequency – Inverse Document Frequency (TF-IDF)
TF-IDF is the multiplication of the TF and IDF which we calculated above.

In [0]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1


,words,tf,idf,tfidf
0,lyft,1,8.762865,8.762865
1,offer,1,6.522155,6.522155
2,cant,1,3.538194,3.538194
3,pdx,1,8.762865,8.762865
4,thanks,1,4.597751,4.597751
5,dont,1,3.745585,3.745585
6,credit,1,7.327781,7.327781
7,van,1,5.236505,5.236505
8,getthanked,1,9.679156,9.679156
9,disapointed,1,10.372303,10.372303


We can see that the TF-IDF has penalized words like ‘don’t’, ‘can’t’, and ‘use’ because they are commonly occurring words. However, it has given a high weight to “disappointed” since that will be very useful in determining the sentiment of the tweet.

We don’t have to calculate TF and IDF every time beforehand and then multiply it to obtain TF-IDF. Instead, sklearn has a separate function to directly obtain it:

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(train['tweet'])

train_vect

<31962x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 114032 stored elements in Compressed Sparse Row format>

In [0]:
train_vect.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

We can also perform basic pre-processing steps like lower-casing and removal of stopwords, if we haven’t done them earlier.

## Bag of Words
Bag of Words (BoW) refers to the representation of text which describes the presence of words within the text data. The intuition behind this is that two similar text fields will contain similar kind of words, and will therefore have a similar bag of words. Further, that from the text alone we can learn something about the meaning of the document.

For implementation, sklearn provides a separate function for it as shown below:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['tweet'])
train_bow

<31962x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 137915 stored elements in Compressed Sparse Row format>

## Sentiment Analysis
If you recall, our problem was to detect the sentiment of the tweet. So, before applying any ML/DL models (which can have a separate feature detecting the sentiment using the textblob library), let’s check the sentiment of the first few tweets.
_

In [0]:
train['tweet'][:5].apply(lambda x: TextBlob(x).sentiment)

id
1    (-0.3, 0.5354166666666667)
2                    (0.2, 0.2)
3                    (0.0, 0.0)
4                    (0.0, 0.0)
5                    (0.0, 0.0)
Name: tweet, dtype: object

Above, you can see that it returns a tuple representing polarity and subjectivity of each tweet. Here, we only extract polarity as it indicates the sentiment as value nearer to 1 means a positive sentiment and values nearer to -1 means a negative sentiment. This can also work as a feature for building a machine learning model.

In [0]:
train['sentiment'] = train['tweet'].apply(lambda x: TextBlob(x).sentiment[0] )
train[['tweet','sentiment']].head()

,tweet,sentiment
id,,
1,father dysfunctional selfish drag kid dysfunct...,-0.3
2,thanks lyft credit cant use cause dont offer w...,0.2
3,bihday majesty,0.0
4,model take urð ðððð ððð,0.0
5,factsguide society motivation,0.0


## Word Embeddings
Word Embedding is the representation of text in the form of vectors. The underlying idea here is that similar words will have a minimum distance between their vectors.

Word2Vec models require a lot of text, so either we can train it on our training data or we can use the pre-trained word vectors developed by Google, Wiki, etc.

Here, we will use pre-trained word vectors which can be downloaded from the glove website. There are different dimensions (50,100, 200, 300) vectors trained on wiki data. For this example, I have downloaded the 100-dimensional version of the model.

You can refer an article here to understand different form of word embeddings.

The first step here is to convert it into the word2vec format.

Now, we can load the above word2vec file as a model.

In [0]:
from gensim.models import KeyedVectors # load the Stanford GloVe model
filename = '/content/drive/My Drive/Shaastra Workshop Material/Text Analysis/GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let’s say our tweet contains a text saying ‘go away’. We can easily obtain it’s word vector using the above model:

In [0]:
model['go']

array([-2.63671875e-02,  6.83593750e-02, -3.11279297e-02,  2.19726562e-01,
        3.41796875e-03, -9.03320312e-03,  1.07910156e-01, -1.74804688e-01,
        7.71484375e-02,  3.83377075e-04, -1.02539062e-01, -1.73339844e-02,
       -3.08837891e-02,  5.76171875e-02, -1.09863281e-01,  6.10351562e-02,
        2.48046875e-01,  5.46264648e-03,  3.49121094e-02,  7.65991211e-03,
       -1.07910156e-01,  2.16796875e-01,  1.26953125e-01,  1.46484375e-01,
        1.55273438e-01,  4.46777344e-02,  7.51953125e-02, -1.45507812e-01,
       -7.71484375e-02, -8.54492188e-02, -1.15966797e-02,  7.95898438e-02,
       -1.94335938e-01, -2.57812500e-01, -9.86328125e-02, -1.38671875e-01,
       -4.13894653e-04, -9.03320312e-02,  7.66601562e-02,  1.33789062e-01,
        5.17578125e-02, -4.80957031e-02,  1.88476562e-01, -3.44238281e-02,
       -7.95898438e-02,  3.54003906e-02, -1.02539062e-01, -1.33789062e-01,
       -1.06445312e-01,  7.42187500e-02, -2.46582031e-02,  1.99218750e-01,
        1.54296875e-01, -

In [0]:
model['away']


array([ 5.88378906e-02, -8.10546875e-02, -7.22656250e-02,  8.78906250e-02,
       -8.64257812e-02, -4.46777344e-02,  3.36914062e-02, -2.81250000e-01,
       -6.64062500e-02,  2.53906250e-01,  6.25000000e-02, -1.90429688e-01,
        3.83300781e-02,  1.91406250e-01, -3.98635864e-04, -4.12597656e-02,
        1.29882812e-01,  9.81445312e-02,  2.41699219e-02, -4.54101562e-02,
        1.12304688e-01,  8.83789062e-02,  1.08398438e-01, -1.28173828e-02,
        3.10546875e-01,  2.56347656e-02,  6.68945312e-02, -1.14257812e-01,
        1.51367188e-01, -2.02148438e-01,  2.04101562e-01,  2.08007812e-01,
       -1.59179688e-01, -9.66796875e-02, -2.08007812e-01,  7.91015625e-02,
       -1.99218750e-01, -9.71679688e-02, -1.33666992e-02,  2.53906250e-01,
        1.20605469e-01, -2.33398438e-01,  1.85546875e-01, -4.10156250e-02,
       -1.71875000e-01,  3.63769531e-02, -8.00781250e-02,  7.51953125e-02,
        4.90722656e-02, -4.49218750e-02, -6.29882812e-02,  9.17968750e-02,
       -3.19824219e-02,  

In [0]:
(model['go'] + model['away'])/2

In [0]:
model.similarity('bed','pillow')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.5638939